In [1]:
import json
import os
from osmxtract import overpass, location
import geopandas as gpd
import requests
import json
shapefile_usosuolo="http://webapps.comune.trento.it/cartografia/gis/dbexport?db=base&sc=uso_suolo&ly=usosuolo_view&fr=shp"
segnalazioni_sensorcivico="https://sensorcivico.comune.trento.it/sensor/data?contentType=geojson"
confini_trento=".."+ os.sep + "data" + os.sep + "confini_trento.shp"
url_dettagli_sensorcivico="https://sensorcivico.comune.trento.it/sensor/data?contentType=marker&id=ID" #sostituire ID con l'ID della segnalazione

In [2]:
#usosuolo=gpd.read_file(shapefile_usosuolo,driver="ESRI Shapefile")

In [3]:
confini = gpd.read_file(confini_trento,driver="ESRI Shapefile")

In [4]:
#aree_verdi_urbane = usosuolo[usosuolo.fumetto=='Aree verdi urbane'].plot()

In [5]:
bounds = (confini.bounds.miny[0],confini.bounds.minx[0], confini.bounds.maxy[0],confini.bounds.maxx[0])

In [6]:
query = overpass.ql_query(bounds,tag='leisure', values=['park'])

In [7]:
response = overpass.request(query)

In [8]:
leisure_park_geojson = overpass.as_geojson(response, 'polygon')

In [9]:
leisure_park_osm = gpd.GeoDataFrame.from_features(leisure_park_geojson)

In [10]:
r = requests.get(segnalazioni_sensorcivico)

In [11]:
data = r.json()

In [27]:
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 17322,
   'properties': {'type': 'segnalazione',
    'name': 'Potatura rami',
    'popupContent': '<em>Loading...</em>'},
   'geometry': {'type': 'Point', 'coordinates': [11.114605, 46.09302]}},
  {'type': 'Feature',
   'id': 17308,
   'properties': {'type': 'reclamo',
    'name': 'manutenzione stradale',
    'popupContent': '<em>Loading...</em>'},
   'geometry': {'type': 'Point', 'coordinates': [11.121338, 46.069051]}},
  {'type': 'Feature',
   'id': 17298,
   'properties': {'type': 'segnalazione',
    'name': 'CONSEGNA NOTE CORTESIA IMIS',
    'popupContent': '<em>Loading...</em>'},
   'geometry': {'type': 'Point', 'coordinates': [11.119298, 46.059833]}},
  {'type': 'Feature',
   'id': 17296,
   'properties': {'type': 'segnalazione',
    'name': 'Bici abbandonata da mesi e mal ridotta',
    'popupContent': '<em>Loading...</em>'},
   'geometry': {'type': 'Point', 'coordinates': [11.124215, 46.071496]}},
  {'type'

In [12]:
for k in range(len(data['features'])):
    data['features'][k]['geometry']['coordinates'][0] = float(data['features'][k]['geometry']['coordinates'][0])
    data['features'][k]['geometry']['coordinates'][1] = float(data['features'][k]['geometry']['coordinates'][1])


In [25]:
poi_sensorcivico = gpd.GeoDataFrame.from_features(data)

In [14]:
park_index = []
poi_index = []
for idxpoi, poi in poi_sensorcivico.iterrows():
    for idxpark, park in leisure_park_osm.iterrows():
        if (poi.geometry.within(park.geometry)):
            park_index.append(idxpark)
            poi_index.append(idxpoi)
            break

In [15]:
len(poi_index)

57

In [16]:
poi_sensorcivico[poi_sensorcivico.index.isin(poi_index)].to_file('poi_su_parchi.geojson', driver="GeoJSON")

In [17]:
poi_sensorcivico

,geometry,name,popupContent,type
0,POINT (11.114605 46.09302),Potatura rami,<em>Loading...</em>,segnalazione
1,POINT (11.121338 46.069051),manutenzione stradale,<em>Loading...</em>,reclamo
2,POINT (11.119298 46.059833),CONSEGNA NOTE CORTESIA IMIS,<em>Loading...</em>,segnalazione
3,POINT (11.124215 46.071496),Bici abbandonata da mesi e mal ridotta,<em>Loading...</em>,segnalazione
4,POINT (11.130475 46.003818),Bidoni dei rifiuti occupano uno stallo,<em>Loading...</em>,reclamo
5,POINT (11.121338 46.069051),Piante da potare,<em>Loading...</em>,segnalazione
6,POINT (11.081707 46.06336),"Accumulo foglie secche, loc. Candriai - parco ...",<em>Loading...</em>,segnalazione
7,POINT (11.121338 46.069051),Uso di alcolici,<em>Loading...</em>,segnalazione
8,POINT (11.12966 46.066576),"Via dei Molini: aiuola, zanzare, pulizia",<em>Loading...</em>,segnalazione
9,POINT (11.143538 46.044769),Lampadina 7° lampione rotta,<em>Loading...</em>,segnalazione


In [26]:
poi_sensorcivico

,geometry,name,popupContent,type
0,POINT (11.114605 46.09302),Potatura rami,<em>Loading...</em>,segnalazione
1,POINT (11.121338 46.069051),manutenzione stradale,<em>Loading...</em>,reclamo
2,POINT (11.119298 46.059833),CONSEGNA NOTE CORTESIA IMIS,<em>Loading...</em>,segnalazione
3,POINT (11.124215 46.071496),Bici abbandonata da mesi e mal ridotta,<em>Loading...</em>,segnalazione
4,POINT (11.130475 46.003818),Bidoni dei rifiuti occupano uno stallo,<em>Loading...</em>,reclamo
5,POINT (11.121338 46.069051),Piante da potare,<em>Loading...</em>,segnalazione
6,POINT (11.081707 46.06336),"Accumulo foglie secche, loc. Candriai - parco ...",<em>Loading...</em>,segnalazione
7,POINT (11.121338 46.069051),Uso di alcolici,<em>Loading...</em>,segnalazione
8,POINT (11.12966 46.066576),"Via dei Molini: aiuola, zanzare, pulizia",<em>Loading...</em>,segnalazione
9,POINT (11.143538 46.044769),Lampadina 7° lampione rotta,<em>Loading...</em>,segnalazione


In [24]:
data['features'][0]['id']

17322